# Visual Document Classifier v2 training

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-ocr-workshop/blob/master/jupyter/VisualDocumentClassifierTraining/SparkOCRVisualDocumentClassifierv2Training.ipynb)

## Set license and AWS keys

Need to specify:
- secret
- license
- aws credentials

### Option #1 - define in this cell

In [1]:
import os

secret = ""
version = secret.split("-")[0]

os.environ['JSL_OCR_LICENSE'] = ""
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""

### Option #2 - provide spark_ocr.json file

In [ ]:
import json, os
import sys

if 'google.colab' in sys.modules:
    from google.colab import files

    if 'spark_ocr.json' not in os.listdir():
      license_keys = files.upload()
      os.rename(list(license_keys.keys())[0], 'spark_ocr.json')

with open('spark_ocr.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

## Install Spark-OCR

It is needed only in case of colab. For other environment you should prepare environment appropriately.

In [3]:
# Installing Dependencies
%pip install --upgrade git+https://github.com/JohnSnowLabs/detectron2.git@frozen_sparkocr
%pip install --upgrade git+https://github.com/JohnSnowLabs/transformers.git@layoutlmv2_onnx

%pip install pyyaml
%pip install datasets==1.18.2

#%pip install spark-ocr==$version --extra-index-url=https://pypi.johnsnowlabs.com/$secret --upgrade
%pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/JohnSnowLabs/detectron2.git (to revision frozen_sparkocr) to /tmp/pip-req-build-mrozd1hc
  Running command git clone --filter=blob:none --quiet https://github.com/JohnSnowLabs/detectron2.git /tmp/pip-req-build-mrozd1hc
  Running command git checkout -b frozen_sparkocr --track origin/frozen_sparkocr
  Switched to a new branch 'frozen_sparkocr'
  Branch 'frozen_sparkocr' set up to track remote branch 'frozen_sparkocr' from 'origin'.
  Resolved https://github.com/JohnSnowLabs/detectron2.git to commit cc87e7ec225b5c0449f47794c03335565ec1120e
  Preparing metadata (setup.py) ... done


You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/JohnSnowLabs/transformers.git (to revision layoutlmv2_onnx) to /tmp/pip-req-build-03xy0yew
  Running command git clone --filter=blob:none --quiet https://github.com/JohnSnowLabs/transformers.git /tmp/pip-req-build-03xy0yew
  Running command git checkout -b layoutlmv2_onnx --track origin/layoutlmv2_onnx
  Switched to a new branch 'layoutlmv2_onnx'
  Branch 'layoutlmv2_onnx' set up to track remote branch 'layoutlmv2_onnx' from 'origin'.
  Resolved https://github.com/JohnSnowLabs/transformers.git to commit 6bb0faa543108a562a73de3bda5a3ea7d23f0fdd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
You should consider upgrading via the '/usr/b

You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Download demo datasets

Here we downloaded demo set. You need to put your images to one folder and prepare labelling txt file as at example.</br>
Instructions here are for the command line, you can also manually download and unzip these files.

In [ ]:
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/datasets/visual_doc_classifier/LayoutLM.v2.voc.txt
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/datasets/visual_doc_classifier/rvl_cdip_tmp_preprocessed.zip
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/datasets/visual_doc_classifier/rvl_cdip_tmp.zip
!unzip rvl_cdip_tmp_preprocessed.zip     
!unzip rvl_cdip_tmp.zip

## Start Spark session with Spark OCR

In [1]:
from sparkocr import start
from pyspark import SparkConf

spark_ocr_jar_path = "../../../target/scala-2.12/"
spark = start(jar_path = spark_ocr_jar_path)

spark

Spark version: 3.3.0
Spark NLP version: 4.0.0
Spark NLP for Healthcare version: 4.0.0
Spark OCR version: 4.0.2rc1



### Define labels

In [2]:
labels = ["advertisement",
          "budget",
          "email",
          "file_folder",
          "form",
          "handwritten",
          "invoice",
          "letter",
          "memo",
          "news_article",
          "presentation",
          "questionnaire",
          "resume",
          "scientific_publication",
          "scientific_report",
          "specification"]

## Option #1: Preprocessing your own data

Images for classification should be placed in one folder ("./rvl_cdip_tmp" in this case)

Labels file should be placed to the same folder. File format is the following. One row - one record, file_path and label separated by space like,

```
file1.jpg 1
file2.jpg 2
```

In [3]:
from sparkocr.transformers import *

df = DatasetReader.readDataset("./rvl_cdip_tmp", spark)
display(df.select("content", "act_label").limit(1))

content,act_label
[49 49 2A 00 34 E...,4


### Repartition your data
To better leverage your cluster you may need repartitioning of your input dataframe

In [4]:
df = df.repartition(8)

In [5]:
from sparkocr.transformers import *
from sparkocr.enums import *
from pyspark.ml import PipelineModel

binary_to_image = BinaryToImage()\
    .setOutputCol("image") \
    .setImageType(ImageType.TYPE_3BYTE_BGR)

img_to_hocr = ImageToHocr()\
    .setInputCol("image")\
    .setOutputCol("hocr")\
    .setIgnoreResolution(False)\
    .setOcrParams(["preserve_interword_spaces=0"])

tokenizer = HocrTokenizer()\
    .setInputCol("hocr")\
    .setOutputCol("token")

# OCR pipeline
pipeline1 = PipelineModel(stages=[
    binary_to_image,
    img_to_hocr,
    tokenizer
])

df = pipeline1.transform(df).cache()
df = df.withColumnRenamed("image", "orig_image")
display(df.select("act_label", "pagenum", "exception", "hocr", "token"))

act_label,pagenum,exception,hocr,token
4,0,null,<div class='ocr...,"[{token, 0, 2, in..."


In [6]:
from sparkocr.utils import get_vocabulary_dict

vocab_file = "LayoutLM.v2.voc.txt"
vocab = get_vocabulary_dict(vocab_file, ",")

doc_class = VisualDocumentClassifierV2() \
    .setInputCols(["token", "orig_image"]) \
    .setOutputCol("label")
doc_class.setVocabulary(vocab)

df = doc_class.getPreprocessedDataset(
  df,
  [1,3,224,224]
  ).cache()

In [7]:
df

orig_image,path,modificationTime,length,act_label,pagenum,hocr,token,input_ids,bbox,image,attention_mask,token_type_ids,exception
{file:/home/jose/...,file:/home/jose/s...,2022-09-06 22:14:30,59658,4,0,<div class='ocr...,"[{token, 0, 2, in...","[101, 7601, 8756,...","[0, 0, 0, 0, 152,...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",null


In [8]:
df.select("path", "input_ids", "bbox", "image", "attention_mask", "token_type_ids", "act_label").write.parquet("preprocessed_dataset")

path,input_ids,bbox,image,attention_mask,token_type_ids,act_label
file:/home/jose/s...,"[101, 7601, 8756,...","[0, 0, 0, 0, 152,...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",4


## Option #2: Use preprocessed datasets
It is possible to load datasets in preprocessed state. You will typically prefer a separate cluster environment to do the preprocessing, as it can take long(a number of hours).
Check this notebook, in the same folder as current one,

Spark-ocr visual doc classifier v2 preprocessing on databricks.ipynb

In [19]:
df = DatasetReader.readPreprocessedDataset("./rvl_cdip_tmp_preprocessed", spark)

display(df.limit(1))

input_ids,bbox,image,attention_mask,token_type_ids,act_label
"[101, 13169, 1051...","[0, 0, 0, 0, 42, ...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",scientific_report


If dataset contains str labels substitute them with int ids

In [20]:
from sparkocr.transformers import *
from pyspark.sql.functions import udf


label2id = {k: v for v, k in enumerate(labels)}
df = df.withColumn('act_label', udf(lambda x: label2id[x])('act_label'))
display(df.limit(1))

input_ids,bbox,image,attention_mask,token_type_ids,act_label
"[101, 13169, 1051...","[0, 0, 0, 0, 42, ...","[255, 255, 255, 2...","[1.0, 1.0, 1.0, 1...","[0, 0, 0, 0, 0, 0...",14


## Training


### Dataframe of preprocessed data
Using either of the above listed options for generating your data, now we will run the training stage.

In [9]:
from sparkocr.transformers import *

trainer = VisualDocumentClassifierV2()
trainer.set_train_param_model_save_path("new_model")
trainer.set_train_param_vocab_path("LayoutLM.v2.voc.txt")
trainer.set_train_param_spark(spark)
trainer.set_train_param_num_train_epochs(2)
trainer.set_train_param_useGPU(False)
trainer.setLabels(labels)

doc_class = trainer.fit(df)

Some weights of the model checkpoint at microsoft/layoutlmv2-base-uncased were not used when initializing LayoutLMv2ForSequenceClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.5.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.0.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.10.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.1.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.0.shortcut.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.21.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.18.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.20.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.0.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.11.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.15.conv2.norm.num_batches

Some weights of LayoutLMv2ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv2-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0


  0%|          | 0/1 [00:00<?, ?it/s]

/home/jose/.local/lib/python3.7/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:781: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  // self.config.image_feature_pool_shape[1]
/home/jose/.local/lib/python3.7/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:791: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='fl

Epoch: 1


  0%|          | 0/1 [00:00<?, ?it/s]

/home/jose/.local/lib/python3.7/site-packages/torch/onnx/utils.py:90: UserWarning: 'enable_onnx_checker' is deprecated and ignored. It will be removed in the next PyTorch release. To proceed despite ONNX checker failures, catch torch.onnx.ONNXCheckerError.
  warnings.warn("'enable_onnx_checker' is deprecated and ignored. It will be removed in "
/home/jose/.local/lib/python3.7/site-packages/torch/onnx/utils.py:103: UserWarning: `use_external_data_format' is deprecated and ignored. Will be removed in next PyTorch release. The code will work as it is False if models are not larger than 2GB, Otherwise set to False because of size limits imposed by Protocol Buffers.
  warnings.warn("`use_external_data_format' is deprecated and ignored. Will be removed in next "
